In [7]:
import minsearch
import json
import os
from tqdm.auto import tqdm


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-tiny"
client = MistralClient(api_key=api_key)

KeyError: 'MISTRAL_API_KEY'

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:23<00:00, 39.64it/s]


In [9]:
query = 'I just disovered the course. Can I still join it?'

In [13]:
def build_prompt(query, search_results):
    """
    Build a prompt for a language model based on a query and search results.

    Parameters:
    query (str): The query or question to be answered.
    search_results (list): List of dictionaries containing 'section', 'question', and 'text' keys.

    Returns:
    str: The formatted prompt string containing the query and relevant context from search results.
    """
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = "\n".join(
        f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n"
        for doc in search_results
    ).strip()

    prompt = prompt_template.format(question=query, context=context)
    return prompt


In [14]:
def llm(prompt):
    """
    Send a prompt to the chat model and return the response.

    Parameters:
    prompt (str): The input text to be sent to the chat model.

    Returns:
    response: The response from the chat model.
    """
    response = client.chat(
        model=model,
        messages=[ChatMessage(role="user", content=prompt)]
    )
    return response.choices[0].message.content


In [10]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [11]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer


In [12]:
rag(query)

NameError: name 'build_prompt' is not defined